Import needed set of packages

In [ ]:
import datetime
import scipy
import pandas as pd
import matplotlib.pyplot as plt
from tools import rednt_tools as tools  # custom helper tools library

Read the data from the CSV files with timestamp as index column

In [ ]:
v_df = pd.read_csv(tools.VIBRATION_FILENAME, sep=tools.SEP, header=None, names=tools.V_HEADER_NAMES,
                   index_col=[0], parse_dates=[0], date_parser=tools.date_parser)

t_df = pd.read_csv(tools.TEMPERATURE_FILENAME, sep=tools.SEP, header=None, names=tools.T_HEADER_NAMES,
                   index_col=[0], parse_dates=[0], date_parser=tools.date_parser)

Make sure that both of the data are sorted

In [ ]:
v_df.sort_values(by='timestamp')
t_df.sort_values(by='timestamp')

Upsample the missing data with the NaNs

In [ ]:
v_df = v_df.resample('{}s'.format(tools.SAMPLE_TIME_SEC)).mean()
t_df = t_df.resample('{}s'.format(tools.SAMPLE_TIME_SEC)).mean()

Merge dataframes based on the closest timestamp and limit to smallest length

In [ ]:
merge_tolerance = datetime.timedelta(seconds=tools.SAMPLE_TIME_SEC)
if len(v_df) > len(t_df):
    df = pd.merge_asof(t_df, v_df, on='timestamp', tolerance=merge_tolerance)
else:
    df = pd.merge_asof(v_df, t_df, on='timestamp', tolerance=merge_tolerance)

Plot the timeseries data

In [ ]:
tools.plot_timeseries_data(df)